<a href="https://colab.research.google.com/github/clagp16/Aprendizaje_automatico_para_pln/blob/main/Practica1_CGP_Terminado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece] -Uq

In [ ]:
from transformers import pipeline
import pandas as pd

##**Clasificación de sentimientos**


In [ ]:
text1 = """Las nuevas tendencias de ropa en Madrid, al igual que en muchas otras partes del mundo, giran en torno a lo que los jovenes denominan 'aesthetics' o 'looks'
, creando de ciertas prendas, tonalidades y complementos, un estilo específico al que la gente se adhiere si con él se siente identificado. """

In [ ]:
text2 = """No me puedo creer que le hayas hecho eso a María. Ella te quiere mucho pero parece que tú no sabes apreciar cuando tienes a una persona buena
y que te quiere a tu lado."""

In [ ]:
text3 = """Ella devoró en la nueva presentación. Qué madre. No me lo esperaba"""


Con los 3 textos anteriores busco ver lo bien que funciona el programa. En teoría, el primero debería categorizarse como neutro, el segundo como negativo y el tercero como positivo.

En el primer texto he incluido neologismos para ver qué sucede. En el segundo he puesto varios palabras 'positivas' como querer, apreciar y buena. Por último, en el texto 3 he usado palabras en tendencia actual que probablemente el programa no reconozca. Estas son devorar = hacer algo muy bien, y madre = mujer (normalmente figura pública), con más de 25 años aproximadamente, que es talentosa y a la que admiras.

In [ ]:
classifier = pipeline(task='text-classification',model='pysentimiento/robertuito-sentiment-analysis')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
outputs = classifier(text1), classifier(text2), classifier(text3)
pd.DataFrame(outputs)

,0
0,"{'label': 'NEU', 'score': 0.500424325466156}"
1,"{'label': 'NEG', 'score': 0.9579558968544006}"
2,"{'label': 'POS', 'score': 0.8467280864715576}"


## **Reconocimiento de entidades**

In [ ]:
ner_taggert = pipeline(model="mrm8488/bert-spanish-cased-finetuned-ner",aggregation_strategy="simple")


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


En otra asignatura vimos que el NER va más allá incluso de las entidades típicas como nombres de personas, paises, organizaciones, etc. Dependiendo de la finalidad debería incluirse también datos como el número de teléfono o el DNI. Mi texto se basará en estos puntos, además de introducir nombres de calles y nombres de organizaciones que no existen, para observar el trabajo del programa.

In [ ]:
outputs = ner_taggert("""Mi amigo Juan perdió ayer su DNI, con número 123456789Q.
Él vive en un bajo en la calle de Alcalá. Trabaja en Atitas. mi hermana maría rosa es ayudante de cocina en un restaurante de atocha.""")
pd.DataFrame(outputs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,entity_group,score,word,start,end
0,PER,0.999168,Juan,9,13
1,MISC,0.998205,D,29,30
2,PER,0.997059,Él,57,59
3,LOC,0.999316,calle de Alcalá,82,97
4,ORG,0.859637,Atitas,110,116


Observamos gracias a los resultados que el modelo parece basarse en las mayúsculas, entre otras cosas. Es por esto que define una 'D' como una entidad del tipo Miscellaneous entities, sin coger realmente el número al que se refiere o las siglas completas 'DNI'. Además, si las palabras empiezan por minúsuclas no las detecta.

## **Respondiendo preguntas**

In [ ]:
text= """Mi abuela ha ido a comprar al super esta mañana y ha visto que no quedaban plátanos, así que ha tenido que ir, después de comprar el resto de productos como detergente
o lechuga, a otra tienda a buscarlos. Después de la primera, visitó dos tiendas más hasta encontrar lo que quería".
"""
question = "¿Qué busca la abuela?"
question2a = "¿En cuántas tiendas ha buscado?"
question2b = "¿En cuántos lugares ha buscado?"
question3 = " ¿Qué productos ha comprado mi abuela?"

Con las preguntas busco ver la capacidad que tiene el programa para inferir conceptos que no se dicen expresamente, como que 'lo que quería', que se menciona en la última línea, son los plátanos que no encontró en el super.

También, quiero hacer la comparación entre la respuesta que daría preguntandole por tiendas (para ver si, aunque solo menciono 2, interpreta super como otra tienda o lo deja aparte) y comparalo con el resultado de 'lugares'.

La pregunta tres busca analizar cuán bien ha entendido el mensaje global de que lo que quería la abuela son plátanos, y que, al final, los ha comprado.

In [ ]:
reader = pipeline(model="mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es")

Exception ignored in: <function tqdm.__del__ at 0x7a4f704d2e80>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/tqdm/std.py", line 1147, in __del__
    def __del__(self):

KeyboardInterrupt: 
Some weights of the model checkpoint at mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [ ]:
outputs = reader(question=question,context=text)
pd.DataFrame([outputs])

,score,start,end,answer
0,0.417202,75,83,plátanos


In [ ]:
outputs2a = reader(question=question2a,context=text)
pd.DataFrame([outputs2a])

,score,start,end,answer
0,0.69948,236,251,dos tiendas más


In [ ]:
outputs2b = reader(question=question2b,context=text)
pd.DataFrame([outputs2b])


,score,start,end,answer
0,0.699452,236,251,dos tiendas más


In [ ]:
outputs3 = reader(question=question3,context=text)
pd.DataFrame([outputs3])

,score,start,end,answer
0,0.940009,157,177,detergente\no lechuga


Observamos que, en general no es capaz de producir ni 'entneder'. Lo que hace es buscar en el texto y no hace ningún tipo de inferencias.

Si el texto cambia ligeramente: 'a otra tienda a por ellos', no llega a ver que se compran los plátanos como tal, y la respuesta a la primera pregunta está incompleta ya que solo incluye el clítico (lo) y no todo el enunciado que se refiere a los plátanos: lo que quería.

Sin emabargo, si el texto es el actual: 'a otra tienda a buscarlos', sí contesta bien a la pregunt ay responde 'plátanos'.

En cuanto a las preguntas número 2 y 3, podemos ver que no hace ninguna distinción y que, auqneu entiende que lugar habla de tiendas, no consigue incluir supermercado en esta categoría.

Con la última pregunta entendemos que no es capaz de producir, solo de copiar, y que no puede ver más allá y hacer inferencias.

Probaremos ahora con el texto en inglés que generará el apartado de traducción para ver si las respuestas son más satisfactorias.

##**Traducción**

In [ ]:
translator_es_en = pipeline(model="Helsinki-NLP/opus-mt-es-en")


/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


In [ ]:
output_text = translator_es_en(text,clean_up_tokenization_spaces=True)
translated_text = output_text[0]['translation_text']
print(translated_text)

My grandmother went to the super this morning and saw that there were no bananas left, so she had to go, after buying the rest of the products like detergent or lettuce, to another store to look for them. After the first one, she visited two more stores until she found what she wanted."


In [ ]:
outputs = translator_es_en(question,clean_up_tokenization_spaces=True)
translated_question = outputs[0]['translation_text']
print(translated_question)

What's Grandma looking for?


In [ ]:
outputs2a = translator_es_en(question2a,clean_up_tokenization_spaces=True)
translated_question2a = outputs2a[0]['translation_text']
print(translated_question2a)

How many stores have you been looking for?


In [ ]:
outputs2b = translator_es_en(question2b,clean_up_tokenization_spaces=True)
translated_question2b = outputs2b[0]['translation_text']
print(translated_question2b)

How many places have you searched?


In [ ]:
outputs3 = translator_es_en(question3,clean_up_tokenization_spaces=True)
translated_question3 = outputs3[0]['translation_text']
print(translated_question3)

What products did my grandmother buy?


Vemos que para las preguntas 2a y 2b, las ha traducido mal y que por ello necesitaríamos incluir el pronombre personal en la pregunta para desambiguar.

In [ ]:
question2a = "¿En cuántas tiendas ha buscado ella?"
question2b = "¿En cuántos lugares ha buscado ella?"

In [ ]:
outputs2a = translator_es_en(question2a,clean_up_tokenization_spaces=True)
translated_question2a = outputs2a[0]['translation_text']
print(translated_question2a)

How many stores has she been looking for?


Esta vuelve a traducirla erroneamente, puesto que no busca las tiendas sino que busca EN tiendas.

In [ ]:
outputs2b = translator_es_en(question2b,clean_up_tokenization_spaces=True)
translated_question2b = outputs2b[0]['translation_text']
print(translated_question2b)

How many places has she searched?


Aquí volvemos a probar el sistema de respondiendo preguntas, pero esta vez con todo el input en inglés.

In [ ]:
outputs1 = reader(question=translated_question,context=translated_text)
pd.DataFrame([outputs1])

,score,start,end,answer
0,0.18716,199,203,them


In [ ]:
outputs2a = reader(question=translated_question2a,context=translated_text)
pd.DataFrame([outputs2a])

,score,start,end,answer
0,0.071793,279,285,wanted


In [ ]:
outputs2b = reader(question=translated_question2b,context=translated_text)
pd.DataFrame([outputs2b])

,score,start,end,answer
0,0.083666,199,203,them


In [ ]:
outputs3 = reader(question=translated_question3,context=translated_text)
pd.DataFrame([outputs3])

,score,start,end,answer
0,0.776145,148,168,detergent or lettuce


Si cabe, los resultados son aún peores. Parece que las 1 y 2b  las responde  de una misma forma y no acaba de apuntar a qué se refiere el clítico que muestra como respuesta. Por otro lado, en la 2b, la respuesta ni siquiera tiene sentido, porque 'them' no refiere en ninguna parte del texto a los lugares en los que busca los plátanos, sino a los plátanos en sí, por lo que es totalmente errónea.

La respuesta a la pregunta 2a es un verbo, por alguna razon, cuando lo que se le está pidiendo en la estructura de la pregunta 'How many stores has she been looking for?', es claramente un número.

La última es la única que parece constestar relativamente de forma correcta.

##**Resumenes**

In [ ]:
summarizer = pipeline(model="josmunpen/mt5-small-spanish-summarization")


Device set to use cpu


In [ ]:
outputs = summarizer(text,max_length=45,clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

Both `max_new_tokens` (=256) and `max_length`(=45) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


mi abuela visita dos tiendas más hasta encontrar lo que quería"


In [ ]:
outputs = summarizer(translated_text,max_length=90,clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

Your max_length is set to 90, but your input_length is only 72. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)
Both `max_new_tokens` (=256) and `max_length`(=90) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


"the grandmother visited a another store to look for more stores"


Los resúmenes tanto en español como en inglés son bastante malos cada uno a su manera.
En español no hace referencia a los plátanos, que es en torno a lo que gira el texto, tampoco hace mención a que la abuela no los encuentra en una primera tienda y es por eso que visita dos tiendas más hasta encontrar lo que quería.

En inglés, para empezar, se pierde que la abuela es mía (utiliza the grandmother en lugar de my grandmother). Además, hay errores gramaticales como en 'visited a another'. Por último, vuelve a tener el mismo error que vimos antes en el que hacía el vínculo entre alguno de los 'them' que aparece en el texto y las tiendas. Por eso a la pregunta 2b responde con them. Aquí, nuevamente, sabe que la abuela visista otras tiendas (aunque en realidad esto también lo malienterpreta y solo menciona una, en singular) en búsqueda de 'them', aunque asocia el valor semántico de este pronombre a un elemento incorrecto.


##**Generador**

In [ ]:
generator = pipeline(model="DeepESP/gpt2-spanish")


Device set to use cpu


In [ ]:
respuesta = "Al final, mi abuela denunció al super por no tener plátanos"
prompt = text + respuesta
outputs = generator(prompt,max_length=300)
print(outputs[0]['generated_text'])

Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Mi abuela ha ido a comprar al super esta mañana y ha visto que no quedaban plátanos, así que ha tenido que ir, después de comprar el resto de productos como detergente
o lechuga, a otra tienda a buscarlos. Después de la primera, visitó dos tiendas más hasta encontrar lo que quería".
Al final, mi abuela denunció al super por no tener plátanos. Las dos estaban muy ocupadas con los plátanos, pero en lugar de ponerlos en la cesta, lo hacía como si fuera la fruta prohibida de la infancia. 

No fue fácil encontrar un almacén para la gente más joven de la ciudad, que estaba a unos ciento cincuenta kilómetros del pueblo. A la derecha, en el aparcamiento de la panadería, había dos puertas, una en cada lado y otra en el centro. Dos de ellas, una de ellas era la entrada a un almacén para el supermercado de la panadería, donde tenían una despensa de madera y unos contenedores de basura. Entre las dos primeras, estaba el almacén de la panadería, en el que se encontraba la panadería. 

"No se me ocu

Mantiene la figura de la abuela pero desvaría y nada parece estar relacionado. Parece que genera una frases, después identifica palabras clave y genera más frases a partir de ellas. Sin emabrgo, el texto carece de cohesión y coherencia general. Ni siquiera todas las frases (en sí) parecen tener estas cualidades.

###***PARTE OPCIONAL***

In [1]:
from transformers import pipeline
classifier = pipeline("text-classification", model="Falconsai/intent_classification")



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


In [15]:
text1 = "Help me understand the concept of atoms"
result1 = classifier(text1)

text2a = "Could you lend me a hand with cooking tomorrow?"
result2a = classifier(text2a)

text2b = "Could you lend me a hand with cooking?"
result2b = classifier(text2b)

text3 = "Why is the earth round?"
result3 = classifier(text3)

text4 = "I wanna book a plane"
result4 = classifier(text4)

text5 = "Annotate the next text semantically"
result5 = classifier(text5)

"""text6 =
result6 = classifier(text6)"""

print('La respuesta a la frase: ', text1, '-->', result1)
print('La respuesta a la frase: ', text2a, '-->',result2a)
print('La respuesta a la frase: ', text2b, '-->',result2b)
print('La respuesta a la frase: ', text3, '-->',result3)
print('La respuesta a la frase: ', text4, '-->',result4)
print('La respuesta a la frase: ', text5, '-->',result5)
#print('La respuesta a la frase: ', text6, '-->',result6)

La respuesta a la frase:  Help me understand the concept of atoms --> [{'label': 'ordering', 'score': 1.0}]
La respuesta a la frase:  Could you lend me a hand with cooking tomorrow? --> [{'label': 'appointment', 'score': 1.0}]
La respuesta a la frase:  Could you lend me a hand with cooking? --> [{'label': 'speak to person', 'score': 1.0}]
La respuesta a la frase:  Why is the earth round? --> [{'label': 'appointment', 'score': 1.0}]
La respuesta a la frase:  I wanna book a plane --> [{'label': 'ordering', 'score': 1.0}]
La respuesta a la frase:  Annotate the next text semantically --> [{'label': 'edit account', 'score': 1.0}]


In [34]:
from transformers import pipeline
classifier = pipeline("image-classification", model="google/vit-base-patch16-224")

oso  = classifier("/content/drive/MyDrive/oso1.jpg")
oso1 = classifier("/content/drive/MyDrive/oso2.jpg")
oso2 = classifier("/content/drive/MyDrive/oso3.jpg")
osopanda = classifier("/content/drive/MyDrive/osopanda.jpg")
osopolar = classifier("/content/drive/MyDrive/osopolar.jpg")

print(oso[0]['label'])
print(oso1[0]['label'])
print(oso2[0]['label'])
print(osopanda[0]['label'])
print(osopolar[0]['label'])

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Device set to use cpu


brown bear, bruin, Ursus arctos
brown bear, bruin, Ursus arctos
brown bear, bruin, Ursus arctos
giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca
ice bear, polar bear, Ursus Maritimus, Thalarctos maritimus
